In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_path = Path("../ML Engineer/train.csv")
test_path = Path("../ML Engineer/test.csv")
submission_path = Path("../ML Engineer/sample_submission.csv")

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

In [3]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
submission_data = pd.read_csv(submission_path)

In [4]:
train_data.head()

,id_,source,email,class
0,5732aa7f-0c44-4a4f-877a-0488aed0d1f7,2,Subject: is the supply rebound beginning ? an ...,not_spam
1,4d3c392d-a4f0-465d-baa3-2c15f1560f07,2,Subject: email list - 100 million addresses $ ...,spam
2,d47e95c0-4909-41b8-aec8-a3fb953fa18f,4,Subject: alley dodecahedra suicide\nare you re...,spam
3,658a83eb-689c-480a-ae31-d622dc83f9f8,6,Subject: ibuyit project\ni wanted to share som...,not_spam
4,179d10b7-1c43-4e10-a0be-18d205b0fe24,4,Subject: cheap vicodin online - us fda pharmac...,spam


In [7]:
train_data['email'][3]

'Subject: ibuyit project\ni wanted to share some great news with you about a major corporate initiative\nwithin ets , called the ibuyit project , that you should know about and be\nlooking forward to . ibuyit is a tool that will make it easier for you to\npurchase the things you need to get your job done and / or to operate our\nbusiness . some key points related to ibuyit are :\n? employees at all levels across ets will be able to access ibuyit on their\ncomputers much like the enron intranet . ibuyit provides an upgrade to the\nexisting b 2 b web - based procurement tool .\n? ibuyit will utilize a new catalog system ( requisite ) that will make it\neasier for you to identify material items and incorporate them into the\nrequisition .\n? ibuyit will not only make it simple for you to buy on - line , but it can also\nserve as your single , convenient source to access other tools and information\nyou use everyday at enron .\nstan horton , rod hayslett , and phil lowry decided back in de

In [ ]:
test_data.head()

In [ ]:
submission_data.head()

## Explaratory Data Analysis

In [ ]:
print(f"Train data shape: {train_data.shape}\n"\
      f"Test data shape: {test_data.shape}\n"\
      f"submission data shape: {submission_data.shape}")

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
submission_data.isnull().sum()

In [ ]:
train_data.columns

In [ ]:
train_data.source.unique(), test_data.source.unique()

In [ ]:
train_data['class'].value_counts()

In [ ]:
# Assuming there's a 'label' column for classification
plt.figure(figsize=(8, 6))
sns.countplot(x='class', data=train_data, palette='coolwarm')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

In [ ]:
len(train_data.email[0])

In [ ]:
import nltk

train_data['email'] = train_data['email'].str.lower()
train_data.dropna(subset=['email'], inplace=True)
train_data['email'] = train_data['email'].str.replace(r'[^\w\s]', '', regex=True)
train_data['email'] = train_data['email'].str.strip()
train_data['email'] = train_data['email'].str.replace(r'\s+', ' ', regex=True)

train_data.email[0]

In [ ]:
import spacy
 
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

def remove_stopwords(text):
    doc = nlp(text)
    # Remove stopwords
    filtered_words = [token.text for token in doc if not token.is_stop]
    # Join the filtered words to form a clean text
    clean_text = ' '.join(filtered_words)
    return clean_text

remove_stopwords(train_data.email[0])

In [ ]:
train_data['email'] = train_data['email'].apply(remove_stopwords)

In [ ]:
train_data['text_length'] = train_data['email'].apply(len)
train_data.head()

In [ ]:
# Plot text length distribution
plt.figure(figsize=(10, 6))
sns.histplot(train_data['text_length'], bins=30, kde=True)
plt.title('Distribution of Text Length')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
train_data[train_data['text_length'] ==train_data['text_length'].min()]

In [ ]:
train_data[train_data['text_length'] ==train_data['text_length'].max()]

In [ ]:
train_data['text_length'].describe()

In [ ]:
test_data['text_length'].describe()

In [ ]:
test_data['text_length'] = test_data['email'].apply(len)
print(test_data.text_length.min())
test_data[test_data['text_length'] ==test_data['text_length'].min()]

In [ ]:
from collections import Counter

# Tokenize the cleaned text
all_words = [word for tokens in train_data['email'].str.split() for word in tokens]
word_freq = Counter(all_words)

# Get the 20 most common words
common_words = word_freq.most_common(20)

# Plot the most common words
words, counts = zip(*common_words)
plt.figure(figsize=(12, 8))
sns.barplot(x=list(counts), y=list(words))
plt.title('Top 20 Most Common Words')
plt.xlabel('Frequency')
plt.ylabel('Words')
plt.show()

In [ ]:
train_data['email'][0]

In [ ]:
from wordcloud import WordCloud

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(all_words))

# Display the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Most Common Words')
plt.show()

In [ ]:
spam = train_data[train_data['class']=='spam']
not_spam = train_data[train_data['class']=='not_spam']

spam_all_words = [word for tokens in spam['email'].str.split() for word in tokens]
not_spam_all_words = [word for tokens in not_spam['email'].str.split() for word in tokens]

In [ ]:
# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(spam_all_words))

# Display the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Common Words for Spam Mails')
plt.show()

In [ ]:
# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(not_spam_all_words))

# Display the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Common Words for Non-Spam Mails ')
plt.show()

TfidfVectorizer
Chi2, correlation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.naive_bayes import MultinomialNB

N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

import seaborn as sns

sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

cv_df.groupby('model_name').accuracy.mean()

In [ ]:
model = LinearSVC()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

The vast majority of the predictions end up on the diagonal (predicted label = actual label), where we want them to be. However, there are a number of misclassifications, and it might be interesting to see what those are caused by:

In [ ]:
from IPython.display import display

for predicted in category_id_df.category_id:
  for actual in category_id_df.category_id:
    if predicted != actual and conf_mat[actual, predicted] >= 10:
      print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], id_to_category[predicted], conf_mat[actual, predicted]))
      display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['Product', 'Consumer_complaint_narrative']])
      print('')

chi square test

model.fit(features, labels)

N = 2
for Product, category_id in sorted(category_to_id.items()):
  indices = np.argsort(model.coef_[category_id])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
  bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
  print("# '{}':".format(Product))
  print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
  print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, target_names=df['Product'].unique()))

# Create our list of punctuation marks
punctuations = string.punctuation

# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

stemming, lemmatization

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))



from gensim.parsing.preprocessing import remove_stopwords
 
# Another sample text
new_text = "The majestic mountains provide a breathtaking view."
 
# Remove stopwords using Gensim
new_filtered_text = remove_stopwords(new_text)
 
print("Original Text:", new_text)
print("Text after Stopword Removal:", new_filtered_text)

In [ ]:
# WORD-COUNT
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
print(df_train[df_train['target']==1]['word_count'].mean()) #Disaster tweets
print(df_train[df_train['target']==0]['word_count'].mean()) #Non-Disaster tweets

In [ ]:
#for word embedding
import gensim
from gensim.models import Word2Vec

# PLOTTING WORD-COUNT
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,4))
train_words=df_train[df_train['target']==1]['word_count']
ax1.hist(train_words,color='red')
ax1.set_title('Disaster tweets')
train_words=df_train[df_train['target']==0]['word_count']
ax2.hist(train_words,color='green')
ax2.set_title('Non-disaster tweets')
fig.suptitle('Words per tweet')
plt.show()

In [ ]:
# CHARACTER-COUNT
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
print(df_train[df_train['target']==1]['char_count'].mean()) #Disaster tweets
print(df_train[df_train['target']==0]['char_count'].mean()) #Non-Disaster tweets

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [ ]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
from nltk.corpus import stopwords
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [ ]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df_train['clean_text'] = df_train['text'].apply(lambda x: finalpreprocess(x))
df_train.head()

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["target"],test_size=0.2,shuffle=True)
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [ ]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
def fit(self, X, y):
        return self
def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
w2v = dict(zip(model.wv.index2word, model.wv.syn0)) df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['clean_text']]
model = Word2Vec(df['clean_text_tok'],min_count=1)     
modelw = MeanEmbeddingVectorizer(w2v)
# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [1]:
import re

text = "Check out my website at https://example.com and follow me on http://twitter.com/username!"

# Regular expression to match URLs
url_pattern = re.compile(r'http[s]?://\S+')

# Substitute URLs with an empty string
text_without_links = url_pattern.sub('', text)

print(text_without_links)


Check out my website at  and follow me on 


Bag-of-Words (with Tf-Idf) and Word Embedding with Word2Vec. You can further enhance the performance of your model using this code by

using other classification algorithms like Support Vector Machines (SVM), XgBoost, Ensemble models, Neural networks etc.
using Gridsearch to tune the hyperparameters of your model
using advanced word-embedding methods like GloVe and BERT

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [4]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)

ModuleNotFoundError: No module named 'spellchecker'

In [ ]:
Globe, Word2Vec , pycaret , 

In [2]:
import re


example="New competition launched :https://www.kaggle.com/c/nlp-getting-started   in our srouse"

In [3]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)

'New competition launched :   in our srouse'

# Deep Learning

In [ ]:
pip install torch torchvision torchaudio
pip install tqdm  # for progress bars
pip install scikit-learn  # for metrics like accuracy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np


# Example dataset
texts = ["I love machine learning", "Machine learning is fascinating", "I love coding in Python", ...]
labels = ["positive", "positive", "neutral", ...]

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize the text data (basic tokenization, use more advanced tokenizers for better performance)
def tokenize(text):
    return text.lower().split()

# Build vocabulary
vocab = set(token for text in X_train for token in tokenize(text))
vocab = {word: i+1 for i, word in enumerate(vocab)}  # index 0 is reserved for padding

# Convert text to sequence of indices
def text_to_sequence(text, vocab):
    return [vocab.get(token, 0) for token in tokenize(text)]

X_train_seq = [text_to_sequence(text, vocab) for text in X_train]
X_test_seq = [text_to_sequence(text, vocab) for text in X_test]

class TextDataset(Dataset):
    def __init__(self, texts, labels, max_len=50):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Pad sequences
        if len(text) < self.max_len:
            text = text + [0] * (self.max_len - len(text))
        else:
            text = text[:self.max_len]
        
        return torch.tensor(text, dtype=torch.long), torch.tensor(label, dtype=torch.long)

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, num_class)
    
    def forward(self, text):
        embedded = self.embedding(text)
        pooled = torch.mean(embedded, dim=1)
        return self.fc(pooled)

# Parameters
vocab_size = len(vocab) + 1  # +1 for padding token
embed_dim = 128
num_class = len(label_encoder.classes_)

# Model instance
model = TextClassificationModel(vocab_size, embed_dim, num_class)

# Hyperparameters
learning_rate = 0.001
batch_size = 32
epochs = 10

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# DataLoader
train_dataset = TextDataset(X_train_seq, y_train)
test_dataset = TextDataset(X_test_seq, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


model.train()

for epoch in range(epochs):
    total_loss = 0
    for texts, labels in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(texts)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')


model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.tolist())
        y_true.extend(labels.tolist())

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

def predict(text, model, vocab, max_len=50):
    model.eval()
    text_seq = text_to_sequence(text, vocab)
    if len(text_seq) < max_len:
        text_seq = text_seq + [0] * (max_len - len(text_seq))
    else:
        text_seq = text_seq[:max_len]
    text_tensor = torch.tensor([text_seq], dtype=torch.long)
    
    with torch.no_grad():
        output = model(text_tensor)
        _, predicted = torch.max(output, 1)
    
    return label_encoder.inverse_transform(predicted.tolist())[0]

# Example prediction
print(predict("I enjoy learning new things", model, vocab))




In [ ]:
from gensim.models.doc2vec import TaggedDocument


# Transformers

In [ ]:
pip install transformers
pip install datasets

from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'Corona_NLP_train.csv', 'test': 'Corona_NLP_test.csv'}, encoding = "ISO-8859-1")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def transform_labels(label):

    label = label['Sentiment']
    num = 0
    if label == 'Positive':
        num = 0
    elif label == 'Negative':
        num = 1
    elif label == 'Neutral':
        num = 2
    elif label == 'Extremely Positive':
        num = 3
    elif label == 'Extremely Negative':
        num = 4

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['OriginalTweet'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(40000, 41000))

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

trainer.train()

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()



In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

def load_imdb_data(data_file):
    df = pd.read_csv(data_file)
    texts = df['review'].tolist()
    labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
    return texts, labels

data_file = "/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
texts, labels = load_imdb_data(data_file)

class TextClassificationDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}
    

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits
    
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()



def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return "positive" if preds.item() == 1 else "negative"

# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BERTClassifier(bert_model_name, num_classes).to(device)

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


   for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

torch.save(model.state_dict(), "bert_classifier.pth")


  # Test sentiment prediction
    test_text = "The movie was great and I really enjoyed the performances of the actors."
    sentiment = predict_sentiment(test_text, model, tokenizer, device)
    print("The movie was great and I really enjoyed the performances of the actors.")
    print(f"Predicted sentiment: {sentiment}")


# Test sentiment prediction
    test_text = "Worst movie of the year."
    sentiment = predict_sentiment(test_text, model, tokenizer, device)
    print("Worst movie of the year.")
    print(f"Predicted sentiment: {sentiment}")


## Word2Vec

In [1]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/24.0 MB 4.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/24.0 MB 4.8 MB/s eta 0:00:05
   --- ------------------------------------ 2.1/24.0 MB 3.0 MB/s eta 0:00:08
   ---- ----------------------------------- 2.9/24.0 MB 3.6 MB/s eta 0:00:06
   ------ --------------------------------- 3.7/24.0 MB 3.5 MB/s eta 0:00:06
   ------ --------------------------------- 4.2/24.0 MB 3.5 MB/s eta 0:00:06
   -------- ------------------------------- 5.0/24.0 MB 3.5 MB/s eta 0:00:06
   --------- ------------------------------ 5.8/24.0 MB 3.4 MB/s eta 0:00:06
   ---------- ----------------------------- 6.3/24.0 MB 3.3 MB/s eta 0:00:06
   ----------- ---------------------------- 6.8/24.0 MB 3.3 MB/s eta 0:00:06
   ------------ --------------------------- 7.3/24.0 MB 3.2 MB/s eta 0:00:06
   ----------

In [2]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import nltk

# Download stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdullah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
sentences = [
    "Natural language processing makes computers understand text and speech.",
    "Machine learning is a subset of artificial intelligence.",
    "Word embeddings capture semantic meanings of words.",
    "Deep learning models are complex and require a lot of data."
]

# Preprocess the sentences (tokenize and remove stopwords)
stop_words = set(stopwords.words('english'))
processed_sentences = [[word for word in simple_preprocess(sentence) if word not in stop_words] for sentence in sentences]
processed_sentences

[['natural',
  'language',
  'processing',
  'makes',
  'computers',
  'understand',
  'text',
  'speech'],
 ['machine', 'learning', 'subset', 'artificial', 'intelligence'],
 ['word', 'embeddings', 'capture', 'semantic', 'meanings', 'words'],
 ['deep', 'learning', 'models', 'complex', 'require', 'lot', 'data']]

In [5]:
# Set parameters for Word2Vec
model = Word2Vec(
    sentences=processed_sentences,
    vector_size=100,  # Dimensionality of the word vectors
    window=5,         # Context window size
    min_count=1,      # Ignores words with total frequency lower than this
    workers=4,        # Number of worker threads to train the model
    sg=0              # Training algorithm: 1 for skip-gram; 0 for CBOW
)

# Train the model
model.train(processed_sentences, total_examples=len(processed_sentences), epochs=10)


(40, 260)

In [6]:
# Get the vector for a word
word_vector = model.wv['language']
print(word_vector)

# Find similar words
similar_words = model.wv.most_similar('language')
print(similar_words)

[-8.2428791e-03  9.2996312e-03 -2.0324520e-04 -1.9623127e-03
  4.6078889e-03 -4.0994231e-03  2.7446921e-03  6.9502438e-03
  6.0624750e-03 -7.5168125e-03  9.3859164e-03  4.6658926e-03
  3.9652535e-03 -6.2409118e-03  8.4593408e-03 -2.1446843e-03
  8.8366689e-03 -5.3645796e-03 -8.1226956e-03  6.8163285e-03
  1.6780979e-03 -2.1975953e-03  9.5146457e-03  9.4916159e-03
 -9.7695924e-03  2.5073413e-03  6.1499067e-03  3.8820067e-03
  2.0254797e-03  4.2475312e-04  6.8322109e-04 -3.8205353e-03
 -7.1395137e-03 -2.0916054e-03  3.9166659e-03  8.8147186e-03
  9.2658531e-03 -5.9704902e-03 -9.4034951e-03  9.7668255e-03
  3.4359887e-03  5.1680701e-03  6.2840963e-03 -2.8036023e-03
  7.3251370e-03  2.8331103e-03  2.8640917e-03 -2.3796128e-03
 -3.1250229e-03 -2.3669479e-03  4.2813178e-03  8.1156031e-05
 -9.5881447e-03 -9.6666887e-03 -6.1505702e-03 -1.3877029e-04
  1.9965856e-03  9.4291028e-03  5.5879811e-03 -4.2947554e-03
  2.7267737e-04  4.9670311e-03  7.7122939e-03 -1.1377098e-03
  4.3169018e-03 -5.80825